
<div style="width: 100%; max-width: 100%; background-color: #fefefe; border: 1px solid #333; border-radius: 10px; padding: 20px; font-family: Arial, sans-serif; box-sizing: border-box;">
  <h3 style="color: #2c3e50; text-align: center;">AR(p) Model</h2>
  
  <p style="color: #34495e; line-height: 1.6;">
  The workflow for this section is:
  
  <p style="color: #34495e; line-height: 1.6;">
  1. Extract GDP, year, and country names.
  <br/>
  2.	Train individual models for each country, calculating the best p-value with minimized MSE. The training set size was fixed at 70% due to the simplicity of the AR model. Training set impacts are discussed in the VAR model section.
  <br/>
  3.	Create an interactive visualization that shows errors instead of MSE for better intuitiveness. Test set errors for each year highlight prediction issues caused by events like COVID-19, which the model cannot effectively account for.

  <p style="color: #34495e; line-height: 1.6;">
  Parallel computing was used to accelerate the process, defaulting to maximum parallelism. However, testing revealed the computational load was lower than expected, making serial computation sufficient.
  </p>
  
</div>

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from joblib import Parallel, delayed
import warnings
from ipywidgets import interact, Dropdown, IntSlider, Layout

warnings.filterwarnings('ignore')

# 1. Load data
data_full = pd.read_csv('data_imputation_full.csv', index_col=[0, 1])
gdp_data = data_full.xs('Economics: GDP', level=1)

# Reshape data from wide to long format
gdp_long = gdp_data.reset_index().melt(id_vars=['Country'], var_name='Year', value_name='GDP')
gdp_long['Year'] = gdp_long['Year'].astype(int)

In [17]:
# 2. Define AR model functions
def fit_ar_p(data, p):
    N = len(data)
    if N <= p:
        raise ValueError("Data length must be greater than lag order p")

    # Construct lag matrix
    A = np.column_stack([np.ones(N - p)] + [data[p - i - 1:N - i - 1] for i in range(p)])
    b = data[p:]
    beta, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    return beta, A, b

def predict_ar_p(history, beta, p):
    if len(history) < p:
        raise ValueError("History length must be greater than lag order p")
    feature = np.array([1] + history[-1:-p-1:-1])
    prediction = np.dot(beta, feature)
    return prediction

def predict_sequence(history, beta, p, steps, use_actual=False, actual_values=None):
    predictions = []
    for i in range(steps):
        hist = history[-p:]
        pred = predict_ar_p(hist, beta, p)
        predictions.append(pred)
        if use_actual and actual_values is not None and i < len(actual_values):
            history.append(actual_values[i])
        else:
            history.append(pred)
    return predictions

# 3. Train AR models and select best p
def train_and_select_p(country, df, p_values, train_ratio=0.7):
    try:
        country_df = df[df['Country'] == country].sort_values('Year')
        gdp = country_df['GDP'].values

        if len(gdp) < max(p_values) + 1:
            print(f"{country}: data points are not enough ({len(gdp)}) for AR model training.\n")
            return {
                'Country': country,
                'Best_p': None,
                'MSE': None,
                'MAE': None,
                'R2': None,
                'Beta': None,
                'Train': None,
                'Test': None,
                'Predictions': None
            }

        # Divide data into training and testing sets
        train_size = int(len(gdp) * train_ratio)
        train, test = gdp[:train_size], gdp[train_size:]

        best_p = None
        best_mse = float('inf')
        best_beta = None
        best_predictions = None

        for p in p_values:
            if len(train) <= p:
                continue
            try:
                # Fit AR(p) model
                beta, _, _ = fit_ar_p(train, p)

                # Predict using actual test data
                history = list(train)
                predictions = []
                for i in range(len(test)):
                    pred = predict_ar_p(history, beta, p)
                    predictions.append(pred)
                    history.append(test[i])  # Use actual test data

                mse = mean_squared_error(test, predictions)

                if mse < best_mse:
                    best_mse = mse
                    best_p = p
                    best_beta = beta
                    best_predictions = predictions
            except Exception as e:
                continue

        # Calculate MAE and R2
        if best_p is not None:
            mae = mean_absolute_error(test, best_predictions)
            r2 = r2_score(test, best_predictions)
        else:
            mae = None
            r2 = None

        return {
            'Country': country,
            'Best_p': best_p,
            'MSE': best_mse if best_p is not None else None,
            'MAE': mae,
            'R2': r2,
            'Beta': best_beta,
            'Train': train,
            'Test': test,
            'Predictions': best_predictions
        }

    except Exception as e:
        print(f"{country}: failed. Error message: {e}\n")
        return {
            'Country': country,
            'Best_p': None,
            'MSE': None,
            'MAE': None,
            'R2': None,
            'Beta': None,
            'Train': None,
            'Test': None,
            'Predictions': None
        }

# Countries and p_values
countries = gdp_long['Country'].unique()
p_values = list(range(1, 21))

# Training in parallel
results = Parallel(n_jobs=-1)(
    delayed(train_and_select_p)(country, gdp_long, p_values) for country in countries
)

# Transform results into DataFrame
results_df = pd.DataFrame(results)

# 4. Evaluate AR models
evaluation_df = results_df.dropna(subset=['Beta', 'Best_p']).reset_index(drop=True)
evaluation_df.to_csv('AR_evaluation.csv', index=False)

In [ ]:
# 5. Plotting Function with Corrected Predictions
def forecast_future_gdp_manual(country, df, beta, p, steps=5):
    try:
        country_df = df[df['Country'] == country].sort_values('Year')
        gdp = country_df['GDP'].values.tolist()

        # Start with the most recent p data points
        history = gdp[-p:]

        predictions = []
        for _ in range(steps):
            pred = predict_ar_p(history, beta, p)
            predictions.append(pred)
            history.append(pred)  # Use predicted value for future forecasting
            history = history[-p:]  # Keep only the last p values

        return predictions

    except Exception as e:
        print(f"{country}: GDP forecast failed. Error message: {e}\n")
        return [None]*steps

def plot_gdp_with_forecast_manual(country, forecast_steps=5):
    try:
        # Get historical data
        country_history = gdp_long[gdp_long['Country'] == country].sort_values('Year')
        years_history = country_history['Year'].values
        gdp_history = country_history['GDP'].values

        test_steps=10

        if len(country_history) <= test_steps:
            print(f"Not enough data points to split into train and test for {country}.")
            return

        # Split into train and test
        train_data = country_history.iloc[:-test_steps]
        test_data = country_history.iloc[-test_steps:]

        train_years = train_data['Year'].values
        train_gdp = train_data['GDP'].values

        test_years = test_data['Year'].values
        test_gdp = test_data['GDP'].values

        # Get AR model parameters for this country
        row = evaluation_df[evaluation_df['Country'] == country]
        if row.empty:
            print(f"{country} not found in evaluation results.\n")
            return

        beta = row['Beta'].values[0]
        p = row['Best_p'].values[0]

        if beta is None or p is None:
            print(f"No AR model parameters found for {country}.\n")
            return

        # Ensure that p is less than the length of train data
        if len(train_gdp) < p:
            print(f"Not enough training data points for p={p} for {country}.")
            return

        # Predict test data points using AR model and actual test data
        history = list(train_gdp)
        test_predictions = []
        for i in range(len(test_gdp)):
            pred = predict_ar_p(history, beta, p)
            test_predictions.append(pred)
            history.append(test_gdp[i])  # Use actual test data

        # Forecast future GDP values
        future_predictions = forecast_future_gdp_manual(country, gdp_long, beta, p, steps=forecast_steps)
        if future_predictions is None or None in future_predictions:
            print(f"Forecasting failed for {country}.\n")
            future_predictions = []

        # Prepare future years
        last_year = years_history.max()
        forecast_years = np.arange(last_year + 1, last_year + 1 + forecast_steps)

        # Compute errors
        errors = test_gdp - np.array(test_predictions)

        # Plot the GDP data with forecasts
        plt.figure(figsize=(12, 6))
        plt.plot(train_years, train_gdp, marker='o', label='Train Data')
        plt.plot(test_years, test_gdp, marker='o', label='Test Data')
        plt.plot(test_years, test_predictions, marker='o', linestyle='--', label='AR Model Predictions')
        if future_predictions:
            plt.plot(forecast_years, future_predictions, marker='o', linestyle='--', label='Forecasted GDP')
        plt.title(f'{country} GDP with AR Model Predictions and Forecast')
        plt.xlabel('Year')
        plt.ylabel('GDP')
        plt.legend()
        plt.grid(True)


        # Plot the errors separately
        plt.figure(figsize=(12, 6))
        plt.bar(test_years, errors)
        plt.title(f'{country} Prediction Errors (Test Data - Predictions)')
        plt.xlabel('Year')
        plt.ylabel('Error')
        plt.grid(True)

    except Exception as e:
        print(f"{country}: Plotting failed. Error message: {e}\n")

def perform_forecast(row, df, steps=5):
    country = row['Country']
    beta = row['Beta']
    p = row['Best_p']

    if beta is None or p is None:
        return {
            'Country': country,
            'Forecast_GDP': [None]*steps
        }

    forecast = forecast_future_gdp_manual(country, df, beta, p, steps)
    return {
        'Country': country,
        'Forecast_GDP': forecast
    }

# Forecast future GDP values
forecast_steps = 10
forecast_results = Parallel(n_jobs=-1)(
    delayed(perform_forecast)(row, gdp_long, steps=forecast_steps) for index, row in evaluation_df.iterrows()
)

# Transform results into DataFrame
forecast_df = pd.DataFrame(forecast_results)
forecast_expanded = pd.DataFrame(forecast_df['Forecast_GDP'].tolist(),
                                 columns=[f'Forecast_GDP_{i+1}' for i in range(forecast_steps)])
forecast_final = pd.concat([forecast_df['Country'], forecast_expanded], axis=1)
forecast_final.to_csv('AR_forecast.csv', index=False)

# 6. Interactive Plotting with Widgets
default_country = 'United States'
default_test_steps = 5
default_forecast_steps = 5

country_dropdown = Dropdown(
    options=sorted(countries),
    description='Country:',
    value=default_country,
    style={'description_width': '120px'},
    layout=Layout(width='400px', margin='0 0 0 100px'),
    disabled=False
)

# test_slider = IntSlider(
#     min=1,
#     max=10,
#     step=1,
#     value=default_test_steps,
#     description='Test Steps:',
#     style={'description_width': '120px'},
#     layout=Layout(width='400px', margin='0 0 0 100px')
# )

forecast_slider = IntSlider(
    min=1,
    max=10,
    step=1,
    value=default_forecast_steps,
    description='Forecast Steps:',
    style={'description_width': '120px'},
    layout=Layout(width='400px', margin='0 0 0 100px')
)

interact(
    plot_gdp_with_forecast_manual,
    country=country_dropdown,
    # test_steps=test_slider,
    forecast_steps=forecast_slider
)
                     

interactive(children=(Dropdown(description='Country:', index=160, layout=Layout(margin='0 0 0 100px', width='4…

<function __main__.plot_gdp_with_forecast_manual(country, forecast_steps=5)>